<a href="https://colab.research.google.com/github/YellaCRE/FIFA_RecommandSystem/blob/main/FIFA_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 피파온라인4 선수추천알고리즘_데이터수집

In [ ]:
#pip install beautifulsoup4
#!pip install tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import urllib
import pandas as pd
from pprint import pprint
import json
from numpy.ma.core import append
from tqdm import tqdm
import re

선수 ID 불러오기

In [ ]:
url = "https://static.api.nexon.co.kr/fifaonline4/latest/spid.json"
result_obj = urllib.request.urlopen(url)
print(result_obj) 

json 파일을 dict으로 변환

In [ ]:
result_json = result_obj.read()
result_dict = json.loads(result_json)

In [ ]:
#pprint(result_dict)

선수 ID 데이터프레임

In [ ]:
player_id = pd.DataFrame(result_dict)
player_id.insert(0,'index',range(len(player_id)))
player_id

,index,id,name
0,0,101000001,데이비드 시먼
1,1,101000195,로비 파울러
2,2,101000240,로이 킨
3,3,101000244,게리 네빌
4,4,101000246,폴 스콜스
...,...,...,...
53068,53068,510255917,고영준
53069,53069,510256252,박찬용
53070,53070,510256685,홍시후
53071,53071,510260498,곽윤호


In [ ]:
player_id.iloc[19999]

index        19999
id       300248814
name       N. 코르마즈
Name: 19999, dtype: object

## 선수 스탯 크롤링

In [ ]:
player_list1 = []

for num in tqdm(range(0,1000)): #range(len(player_id)): 여기서 선수 id를 통해 범위를 수정하면 됨
  spid = str(player_id.iloc[num,1])
  url = 'https://fifaonline4.nexon.com/DataCenter/PlayerInfo?spid='+ spid + '&n1Strong=1'

  webpage = requests.get(url)
  soup = bs(webpage.content, "html.parser")

  season = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_name > div.season > img')
  position = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_ab')
  height = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_etc > span.etc.height')
  weight = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_etc > span.etc.weight')
  stat1 = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_bottom > ul:nth-of-type(1)')
  stat2 = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_bottom > ul:nth-of-type(2)')
  stat3 = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_bottom > ul:nth-of-type(3)')
  stat4 = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_bottom > ul:nth-of-type(4)')
  stat5 = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_bottom > ul:nth-of-type(5)')
  skill = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_etc > span.etc.skill > span')
  specif = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.skill_wrap')
  pay = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_name > div.side_utils > div')
  foot = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_etc > span.etc.foot')
  physical = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_etc > span.etc.physical')
  nation = soup.select('#middle > div > div > div:nth-of-type(2) > div.content.data_detail > div > div.content_header > div.info_wrap > div.info_line.info_team > div.etc.nation > span.txt')
  club = soup.select('#middle > div > div > div:nth-of-type(4) > div:nth-of-type(1) > div.content.data_detail_club > div.data_table > ul')
  ranker = soup.select('#middle > div > div > div.ranker_record > div.content > div > div.tbody > div')
  

  list1 = []

  #print(str(season[0]).split('/')[-2][:-5])
  list1.append(str(season[0]).split('/')[-2][:-5])
 
  #print(position[0].text)
  list1.append(position[0].text.split()[0])

  #print(height[0].text)
  list1.append(height[0].text)

  #print(weight[0].text)
  list1.append(weight[0].text)

  #print(stat1[0].text)
  for i in [1,3,6,9,12,15,17]:
    list1.append(stat1[0].text.split()[i])
  for i in [2,5,7,9,12,14,16]: 
    list1.append(stat2[0].text.split()[i])
  for i in [1,4,6,8,11,14,16]:
    list1.append(stat3[0].text.split()[i])
  for i in [1,3,6,8,10,12,14]:  
    list1.append(stat4[0].text.split()[i])
  for i in [1,4,7,10,13,16]:
    list1.append(stat5[0].text.split()[i])

  #print(skill[0].text.count('★'))
  list1.append(skill[0].text.count('★'))

  #print(specif[0].text.replace(" ","").replace("\r\n","")[4:-1]) #벡터화할꺼는 한문장으로 냅두기
  list1.append(specif[0].text.replace(" ","").replace("\r\n","")[4:-1])

  #print(pay[0].text)
  list1.append(pay[0].text)

  #print(foot[0].text.replace(" ","").replace("\r\n","").replace("\n",""))
  list1.append(foot[0].text.replace(" ","").replace("\r\n","").replace("\n",""))

  #print(physical[0].text)
  list1.append(physical[0].text)

  #print(nation[0].text)
  list1.append(nation[0].text)

  #print(club[0].text.replace(" ","").replace("\n",""))
  list1.append(club[0].text.replace(" ","").replace("\n",""))

  #print(ranker[0].text.replace(" ","").replace("\r\n","")[1:-1].split())
  list1.extend(ranker[0].text.replace(" ","").replace("\r\n","")[1:-1].split())

  player_list1.append(list1)


100%|██████████| 1000/1000 [19:27<00:00,  1.17s/it]


데이터프레임으로 변환

In [ ]:
col_name = ['시즌','포지션/스탯', '키', '몸무게', '속력', '가속력', '골 결정력', '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스', '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙', 'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정','개인기', '특성', '급여', '주발/약발', '체형', '국적', '클럽경력','출전','득점','도움','유효슈팅','일반슈팅','패스성공률','드리블성공률','공중볼경합성공률','가로채기','태클성공률','차단성공률','선방','평점']
player_stats = pd.DataFrame(player_list1, columns = col_name)
player_stats.insert(0,'index',range(len(player_stats)))
player_stats.head()

,index,시즌,포지션/스탯,키,몸무게,속력,가속력,골 결정력,슛 파워,중거리 슛,...,유효슈팅,일반슈팅,패스성공률,드리블성공률,공중볼경합성공률,가로채기,태클성공률,차단성공률,선방,평점
0,0,ICON,GK101,193cm,93kg,54,70,48,45,39,...,0.0(0),0.0(0),81.8%9/11,100%1/1,0%0/0,0.0(0),0%0/0,0%0/0,1.6(8),6.3
1,1,ICON,ST101,175cm,72kg,101,104,104,104,101,...,1.0(7),0.0(0),97.4%37/38,85.7%30/35,0%0/5,0.4(3),100%3/3,0%0/0,0.1(1),7.2
2,2,ICON,CDM105,180cm,76kg,95,93,92,89,98,...,0.1(5),0.1(7),92.6%665/718,91.4%594/650,18.6%13/70,0.6(50),67.9%38/56,11.1%13/117,0.5(46),6.2
3,3,ICON,RB102,179cm,79kg,102,100,66,88,56,...,0.1(3),0.1(4),95.6%521/545,95.3%450/472,1.9%1/52,0.6(32),68.3%28/41,9.1%4/44,0.5(29),6.4
4,4,ICON,CM105,171cm,71kg,96,95,102,112,114,...,0.5(63),0.2(19),"91.5%1,480/1,618","91.7%1,375/1,500",3.6%5/140,0.7(82),72.6%77/106,14%6/43,0.2(28),6.8


## csv로 변환

In [ ]:
player_df = pd.merge(player_id,player_stats,on='index')
player_df.head()

,index,id,name,시즌,포지션/스탯,키,몸무게,속력,가속력,골 결정력,...,유효슈팅,일반슈팅,패스성공률,드리블성공률,공중볼경합성공률,가로채기,태클성공률,차단성공률,선방,평점
0,0,101000001,데이비드 시먼,ICON,GK101,193cm,93kg,54,70,48,...,0.0(0),0.0(0),81.8%9/11,100%1/1,0%0/0,0.0(0),0%0/0,0%0/0,1.6(8),6.3
1,1,101000195,로비 파울러,ICON,ST101,175cm,72kg,101,104,104,...,1.0(7),0.0(0),97.4%37/38,85.7%30/35,0%0/5,0.4(3),100%3/3,0%0/0,0.1(1),7.2
2,2,101000240,로이 킨,ICON,CDM105,180cm,76kg,95,93,92,...,0.1(5),0.1(7),92.6%665/718,91.4%594/650,18.6%13/70,0.6(50),67.9%38/56,11.1%13/117,0.5(46),6.2
3,3,101000244,게리 네빌,ICON,RB102,179cm,79kg,102,100,66,...,0.1(3),0.1(4),95.6%521/545,95.3%450/472,1.9%1/52,0.6(32),68.3%28/41,9.1%4/44,0.5(29),6.4
4,4,101000246,폴 스콜스,ICON,CM105,171cm,71kg,96,95,102,...,0.5(63),0.2(19),"91.5%1,480/1,618","91.7%1,375/1,500",3.6%5/140,0.7(82),72.6%77/106,14%6/43,0.2(28),6.8


csv로 변환

In [ ]:
player_df.to_csv("player_dfTest.csv", mode='w', index=False, encoding="utf-8-sig")